# Sedaro Studies Override Tutorial Notebook 
Demonstrates Sedaro Studies support via the override features and the sedaro python client using a Jupyter notebook.

# Introduction  
This notebook explains the override features of the new Studies support added to Sedaro. 
The core of this support is the ability to change starting values of Agent Paramaters of a scenario branch in order to observe its effect on simulation results.

# Setup
Running this notebook requires the following:
- A Sedaro Account            --> https://www.sedaro.com
- A Sedaro API Token          --> https://www.sedaro.com/#/account
- The Wildfire Demo Branch ID --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire Scenarios --> copy main branch ID via clipboard icon (TODO insert screen shot)
- Python 3.10+ installed      --> https://www.python.org
- Jupyter notebook or lab     --> https://jupyter.org

## Pip requirements
Create/activate a python venv if desired

In [1]:
#!python -m venv /path/to/new/virtual/environment

Activate it via this table
| Platform | Shell | Command to activate virtual environment |
| :- | :- | :- |
| POSIX | bash/zsh |  source <venv>/bin/activate |
| | fish |  source <venv>/bin/activate.fish |
| | csh/tcsh |  source <venv>/bin/activate.csh |
| | PowerShell |  <venv>/bin/Activate.ps1 |
| Windows | cmd.exe | C:\> <venv>\Scripts\activate.bat |
| | PowerShell | PS C:\> <venv>\Scripts\Activate.ps1 |

Required python modules:
> replace *python* with *python3* if needed 

In [2]:
#!python -m pip install -e sedaro pytest matplotlib pandas sweetviz

or

In [3]:
#!python3.10 -m pip install  sedaro pytest matplotlib pandas sweetviz

In [4]:
#!python3.10 -m pip install sweetviz IProgress

Run the following cells to test if all required python modules are installed correctly

Optional module needed by the stats() function if desired 

In [1]:
import sedaro
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np

In [2]:
# optional
import sweetviz as sv

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.



## Sedaro python client setup
Note: More information about the sedaro-python client can be found here: [https://github.com/sedaro/sedaro-python]


In the next cell, adjust the following variables as needed
- *Sedaro_api_host*
- *Sedaro_api_token*

In [3]:
Sedaro_api_host  = "http://localhost:80" # "api.sedaro.com"

# Set your API token value either directly or via loading a secrets file
# !! NOTE !!  Be careful not to check in your API Key into a source control repo  !! Note !!
secretPath = '/Users/sedaro/Documents/sedaro/sedaro-satellite/secrets.json'
with open(secretPath, 'r') as file:
    Sedaro_api_token = json.load(file)['API_KEY']
    
sedaroAPI = sedaro.SedaroApiClient(api_key=Sedaro_api_token, host=Sedaro_api_host)

## Load the wildfire scenario branch data
Change the value of the *scenario_branch_id* in the next cell to the branch id noted above during the **Setup** section.
then run the following cell:

In [4]:
scenario_branch_id = "PLS7ksR9fHWl98HnYPZRK8"
wildfire_scenario_branch = sedaroAPI.scenario(scenario_branch_id)

# Overview of the overrides feature
Override objects provide a means of varying Agent model parameters in a scenario to explore trade spaces and result sensitivities in your Agent design.  They can be defined as JSON or Python objects.

Summary of what do override objects do?
- Change starting values of model parameters via selected function(s)
  - Saves these model parameter changes so they can be loaded and re-run later (called snapshots) 
- Available functions include 
  - random draws from various distributions types
  - basic math operators
  - table lookups
  - variable support
  - value clamping
*Note: The **Function** random choices are deterministic between simulation runs.*



Override objects consist of a:
 - **path** to the Agent model parameter to change and a 
 - **function** to perform the actual change itself.

> Note: *Variables* can be used to hold a value of a model parameter or a constant. They are used to ease modification of override values and provide a means to ensure overall model consistency.

The **functions** used to change Agent model parameters support trade-space tables and randomized assignment via functions available in Pythons Random module.

In [6]:
# The following code will list all the override sets currently defined for the scenario branch
[ (overrideset.name, overrideset.id) for overrideset in wildfire_scenario_branch.OverrideSet.get_all()]

[('Tradespace Example', 'PKghWNfbXwHLq8xBDPKsZg')]

## Paths
An addressing means to access a model parameter of an Agent. Currently there are two versions:
- "**path**": A human readable format based on the names of models and its components. 
 - Format: Agent Name / blockname / parameter name / sub series
 - Example: “Wildfire/Crosslink/minTimeBetweenOccurrences/min”
 - Uses “/“ as a delimiter to mimic file location paths
- "**agent_key**": Based on the ids of a model and its components.
 - Format: agent_id.data.blocks.block_id.parameter.subseries
 - Example: “NT0LWIfSJ1RIenKpUmJEV.data.blocks.NV0bIfUUj9e6l-PX4ql1V.loadDefParams.power”
 - Uses ‘.’ for the separator
 - Similar to how Sedaro results streams are named

## Variables
Used to ensure consistency when assigning random values to parameters. When a model parameter is set with the result of a random fn, its value can be saved to a variable in order to update other model parameters that are affected by the change. 

> For example if the area of a solar panel is randomly assigned via a selected distribution function, then the mass of the panel and the Agents overall mass, will need to be adjusted accordingly.

They are initially defined in the variable section of a JSON or Python Override Object. 

### Built-in Variables
Always available to use as values for function arguments

| Name | Description |
| :-: | :- |
| x |  Replaced by the value in the parameter path before the function is run |
| original | Same as 'x' but more explicit |

### User defined
Defined in the "variables" section of the overrides datastructure

The following python command will list all currently defined variables for the first OverrideSet as pydantic json objects

In [7]:
wildfire_scenario_branch.OverrideSet.get('PKghWNfbXwHLq8xBDPKsZg').variables

[{'agent_key': None,
  'equals': None,
  'name': 'Baseline_mass',
  'path': 'Wildfire/data/mass'},
 {'agent_key': None,
  'equals': 2.2,
  'name': 'SAR-10237_mass_delta',
  'path': None},
 {'agent_key': None,
  'equals': 3.2,
  'name': 'SAR-10239_mass_delta',
  'path': None},
 {'agent_key': None,
  'equals': -0.702,
  'name': 'P33081_mass_delta',
  'path': None},
 {'agent_key': None,
  'equals': None,
  'name': 'Baseline_dry_inertia_X.X',
  'path': 'Wildfire/data/inertia/0/0'},
 {'agent_key': None,
  'equals': None,
  'name': 'Baseline_dry_inertia_Y.Y',
  'path': 'Wildfire/data/inertia/1/1'},
 {'agent_key': None,
  'equals': None,
  'name': 'Baseline_dry_inertia_Z.Z',
  'path': 'Wildfire/data/inertia/2/2'}]



or defined as a list of json objects

``` json
variables = [
    {
        "name": "gyroHotTemp",
        "path": "Wildfire/Gyro/hotTempRating/degC",      
    },
    {
        "name": "gyro_Sandwich",
        "equals": "tasty"
    }
]
```

Variables objects must have a 'name' and either:
- A **path** or **Agent_path** key which assigned the variable to the parameters current value
- or A **equals** key which sets the variable to a given value.
 - This value must be the same type of the parameter, aka string, int, or float
  - A key: value pair is used for paths with sub series
   - Example: { "Degrees": 90 } 
 - Values in arrays or lists must have the index in the path address
   - Example: 
     - */Wildfire/data/inertia/0* is the first element of the model parameter inertia list aka **inertia[0]**

## Override Functions
A function executed with provided arguments and stores the result in the model parameter listed in the path key.

### Available override functions
From basic arithmetic to drawing random numbers from a distribution, Sedaro provides a number of built-in functions for use to set starting values of Agent model parameters. 

#### Fn Summary Table
| Basic Math Operators | | Python | Random Module |   |
| :-: | -|  - | - | - |
| + | | choices | sample | randrange | 
| - | | uniform |triangular |expovariate | 
| * | | gammavariate | gauss | normalvariate | 
| / | | lognormvariate | vonmisesvariate | 
| = | | paretovariate | weibullvariate |


#### Basic math operators +, -, *, /, =
Performs the math operator with the current value of the model parameter then stores the results on the given parameter

Examples:
```
{
    "path": "Wildfire/Gyro/hotTempRating/degC",
    "fn": "=",
    "arg": 90.0
},
```
- Sets the model parameter *Wildfire/Gyro/hotTempRating/degC* to 90.0, overriding its original value of 100.0

```
{
   "path": "Wildfire/Power Processor/thermalCapacitance",
   "fn": "*",
   "arg": 1.2
}
```
- Will multiply the original value of the *Wildfire/Power Processor/thermalCapacitance* model parameter by 1.2
- Then it will set the result to the *Wildfire/Power Processor/thermalCapacitance* model parameter

#### Python random module functions
Allows the use of the functions available in pythons random module. These include various random number draws from various types of distribution curves, choices from a list with weights, and so on. More information about these functions can be found here: [https://docs.python.org/3/library/random.html]

---
- Functions for integers
  - Randrange(start, stop[, step]) 
    - generates a random integer within the specified range

---
- Functions for sequences
  -  Choices(population, weights=None, *, cum_weights=None, k=1) 
    - Return a k sized list of elements chosen from the population with replacement. 
  -  Sample(population, k, *, counts=None) 
    - Return a k length list of unique elements chosen from the population sequence. Used for random sampling without replacement.

---
- Real-valued distributions
  - Uniform(a, b) 
    - generates a random float between a and b
  - Triangular(low, high, mode) 
    - Return a random floating point number N such that low <= N <= high
  - Expovariate(lambd=1.0) 
    - Exponential distribution. lambd is 1.0 divided by the desired mean. It should be nonzero.
  - gammavariate(alpha, beta) 
    - Gamma distribution. (Not the gamma function!) The shape and scale parameters, alpha and beta, must have positive values.
  - gauss(mu=0.0, sigma=1.0) 
    - Normal distribution, also called the Gaussian distribution. mu is the mean, and sigma is the standard deviation. 
  - normalvariate(mu=0.0, sigma=1.0) 
    - Normal distribution. mu is the mean, and sigma is the standard deviation.
  - lognormvariate(mu, sigma) 
    - Log normal distribution. If you take the natural logarithm of this distribution, you’ll get a normal distribution with mean mu and standard deviation sigma. 
  - vonmisesvariate(mu, kappa) 
    - mu is the mean angle, expressed in radians between 0 and 2*pi, and kappa is the concentration parameter, which must be greater than or equal to zero.
  - paretovariate(alpha) 
    - Pareto distribution. alpha is the shape parameter.
  - weibullvariate(alpha, beta) 
    - Weibull distribution. alpha is the scale parameter and beta is the shape parameter.
---

Examples:
```
{
    "path": "Wildfire/Battery/initialSoc", # starts as 0.8.
    "fn": "choices",
    "population": [0.6, 0.9], 
    "weights":[1.0, 0.5], 
    "k":1
},
```
Will run the python *random.choices* function with **[0.6, 0.9]** as its *population* and **[1.0,0.5]** as its *weights* arguements
and sets the Agent Model parameter **Wildfire/Battery/initialSoc** to the result.


```
{
    "path": "Wildfire/-Z Surface/temperature/degC", # 20.0
    "fn": "normalvariate",
    "mu": "x", 
    "sigma": 2.0 
},
```
Runs the python *random.normalvariate* function with the arguement *sigma* set to **2.0** and the *mu* arguement set to the current Agents Model parameter **'Wildfire/-Z Surface/temperature/degC'** value. 
> In this example: If the current parameter "Wildfire/-Z Surface/temperature/degC" value was **20.0**, then the *'x'* is replaced by **20.0** which is then used as the *mu* argument for the function call.


##### Function Signatures
Varies, based on the signature of the python random module itself. See the Signatures addendix for details. 

Example:
The python random module function "WeibullVariate" has the following signature:
```
class WeibullVariate(PathFn):
    fn: Literal["weibullvariate"]
    alpha: int | float 
    beta:  int | float 
```
This states the WeibullVariate function expects its alpha and beta parameters to be an integer, float, or string
> Do note arguments can be set with *str* types in this context of defined variable names which are replaced by the variable value before running the function. 


### fn_chain
Performs the provided list of fn overrides in order, each time storing the result in the given parameter path  

Signature:
```
class FnChain(PathFn):
    fn_chain: List[ possible_fns ]
```

Example:
Given the following **fn_chain** override json
```
{
   "path": "Wildfire/Power Processor/thermalCapacitance",
   "fn_chain": [
          {"fn": "*", "arg": 1.2}, 
          {"fn": "-", "arg": 1.0}, 
          {"fn": "triangular", "low": 4000, "high": 4500, "mode": "x" },
          {"fn": "clamp", "low": 4100} ]
}
```
- Will first multiply the value located in the *thermalCapacitance* parameter of the *Wildfire/Power Processor* model by **1.2**
- Then it will substract **1.0** from the previous result
- Then draw a random value from a *triangular* shaped distribution centered around the **current result** *(x is replaced by the current result)*, with low of **4000** and the high of **4500**
- Then clamp the final result to be **4100** or greater
- the final result is then stored in the *Wildfire/Power Processor/thermalCapacitance* parameter

### sim_index
Used to perform tradespace studies. The StudyJob object will create SimJobs each with an unique index. The **sim_index** override will use this index to select from a list of **fn** overrides to use for that SimJob.

The **sim_index**  is modulated by the size of the provided override fn list
> fn used = fn_list[ int(sim_index)%len(fn_list) ]

Signature
``` python
class SimIndex(PathFn):
    sim_index: List[ possible_fns ]
```
Example:
Given the following **sim_index** entry JSON 

``` json
{
  "path": "Wildfire/Power Processor/thermalCapacitance",
  "sim_index": [ 
      {"fn": "uniform",
        "a": 2900.0, 
        "b": 3100  },
      {"fn": "=", "arg": 3000.0}, 
      {"fn": "*", "arg": 1.2}, 
  ]
}
```
Assuming a study with 3 iterations, 
- the first Simulation Job will pull a random value from the uniform distrubation from 2900.0 to 3100.0 
  - and set the **Power Processor/thermalCapacitance** model parameter to its result.
- The second Simulation will directly set the **Power Processor/thermalCapacitance** model parameter to 3000.0.
- The third Simulation will multiply the current  **Power Processor/thermalCapacitance** model parameter by 1.2 and set the parameter value to the result.


### copy_value_to (*Working name*)
Copies the current value of a model parameter into a variable. 

Example:
``` json
{
    "path": "Wildfire/Gyro/hotTempRating/degC",
    "copy_value_to": "gyroHotTemp"          
},
```

Will place the value of the **Wildfire/Gyro/hotTempRating/degC** model parameter into the **gyroHotTemp** variable.

In [9]:
# or TODO Not possible. Only expose OverrideSet currrently
# copy_value_override = wildfire_scenario_branch.CopyValueTo.create(path="Wildfire/Gyro/hotTempRating/degC", 
#                                          copy_value_to="gyroHotTemp")

### clamp
Bounds a parameter value to a given range. Meant to be used with a fn_chain to ensure a randomly set value is within valid bounds.

Example:
``` json
{
    "path":"Wildfire/Power Processor/thermalCapacitance",
    "fn": "clamp", "low": 4100 
},
```
Will set the model parameter **Wildfire/Power Processor/thermalCapacitance** to 4100 if its current value is below the *low* value of 4100

### clampRepeat
Will rerun a given function up to 5 times if the result is outside a given range. Meant to be used with random based override functions.

Example:
``` json
{ 
   
   "path": "LaserComm-1/Quiescent/loadDefParams/power",
   "fn": "clampRepeat",
   "low": 4,
   "high": 5,
   "overrideFn": {
      "fn": "weibullvariate",
      "alpha": 4.5,
      "beta": 1.2,
   }
}
```
Will run the random **weibullvariate** distribution function up to 5 times until a result between 4 and 5 is returned, then it sets the model parameter *LaserComm-1/Quiescent/loadDefParams/power* to it. 

Note: the model parameter will remain its original value if the result clamp [low high] range requirement is not met.

# Example Override Json From the Wildfire tradespace study notebook

This example first defines the variables used in tradespace, then uses them in the override section. The overrides functions shown include basic math, sim_index, and fn_chains of basic math commands.    

In [ ]:
tradespace_overrides_dict = {
  "name": "Tradespace Example",
  "variables": [
    {
      "name": "Baseline_mass",
      "path": "Wildfire/data/mass"
    },
    {
      "name": "SAR-10237_mass_delta",
      "equals": 2.20
    },
    {
      "name": "SAR-10239_mass_delta",
      "equals": 3.20
    },
    {
      "name": "P33081_mass_delta",
      "equals": -0.702
    },
    {
      "name": "Baseline_dry_inertia_X.X",
      "path": "Wildfire/data/inertia/0/0"
    },
    {
      "name": "Baseline_dry_inertia_Y.Y",
      "path": "Wildfire/data/inertia/1/1"
    },
    {
      "name": "Baseline_dry_inertia_Z.Z",
      "path": "Wildfire/data/inertia/2/2"
    }
  ],
  "overrides": [
    {
      "path": "Wildfire/data/mass",
      "sim_index": [ 
                      {"fn": "=", "arg": "Baseline_mass"}, 
                      {"fn": "+", "arg": "SAR-10237_mass_delta"}, 
                      {"fn": "+", "arg": "SAR-10239_mass_delta"}, 
                      {"fn": "+", "arg": "P33081_mass_delta"}, 
                  ]
    },
    {
      "agent_key": "NT06aqHUT5djI1_JPAsck.data.inertia.0.0",
      "sim_index": [ 
                      {"fn": "=", "arg": "Baseline_dry_inertia_X.X"}, 
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10237_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_X.X" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10239_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_X.X" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "P33081_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_X.X" } ] 
                      },
                  ]
    },
    {
      "agent_key": "NT06aqHUT5djI1_JPAsck.data.inertia.1.1",
      "sim_index": [ 
                      {"fn": "=", "arg": "Baseline_dry_inertia_Y.Y"}, 
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10237_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Y.Y" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10239_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Y.Y" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "P33081_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Y.Y" } ] 
                      },
                  ]
    },
    {
      "agent_key": "NT06aqHUT5djI1_JPAsck.data.inertia.2.2",
      "sim_index": [ 
                      {"fn": "=", "arg": "Baseline_dry_inertia_Z.Z"}, 
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10237_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Z.Z" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "SAR-10239_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Z.Z" } ] 
                      },
                      { "fn_chain": [
                          {"fn": "/", "arg": "Baseline_mass"}, 
                          {"fn": "*", "arg": "P33081_mass_delta"}, 
                          {"fn": "+", "arg": "Baseline_dry_inertia_Z.Z" } ] 
                      },
                  ]
    },
  ]
}

This example is from the Monte carlo example notebook and demostrates many of the random-based override functions. 

In [ ]:
# TODO copy over montecarlo example

# Appendix: Override fn Signatures


fn Random module Signatures:

## Functions for integers
  - class Randrange(PathFn):
    - fn: Literal["randrange"]
    - start: int | float | str
    - stop:  int | float | str
    - step:  int | float | str 

---

## Functions for sequences
- class Choices(PathFn):
  - fn: Literal["choices"]
  - weights: list
  - population: list   
  - k: int | str
- class Sample(PathFn):
  - fn: Literal["sample"]
  - counts: list
  - population: list
  - k: int

---

## Real-valued distributions
  - class Uniform(PathFn):
    - fn: Literal["uniform"]
    - a: float | int | str
    - b: float | int | str
  - class Triangular(PathFn):
    - fn: Literal["triangular"]
    - low:  int | float | str
    - high: int | float | str
    - mode: int | float | str
  - class Expovariate(PathFn):
    - fn: Literal["expovariate"]
    - lambd: float | str
  - class GammaVariate(PathFn):
    - fn: Literal["gammavariate"]
    - alpha: int | float | str
    - beta:  int | float | str
  - class Gauss(PathFn):
    - fn: Literal["gauss"]
    - mu:    int | float | str
    - sigma: int | float | str
  - class NormalVariate(PathFn):
    - fn: Literal["normalvariate"]
    - mu:    int | float | str
    - sigma: int | float | str
  - class LogNormVariate(PathFn):
    - fn: Literal["lognormvariate"]
    - mu:    int | float | str
    - sigma: int | float | str
  - class VonmisesVariate(PathFn):
    - fn: Literal["vonmisesvariate"]
    - mu:    int | float | str
    - kappa: int | float | str
  - class ParetoVariate(PathFn):
    - fn: Literal["paretovariate"]
    - alpha: int | float | str    
  - class WeibullVariate(PathFn):
    - fn: Literal["weibullvariate"]
    - alpha: int | float | str
    - beta:  int | float | str